In [33]:
import sys

sys.path.append( '../slevelsutil/' )

import df_util, file_util
from Instance import Instance

### 2) Loading the instance data

In [34]:
filter_instance = {"instance_name":"SCENARIO"} #, "customer_segmentation":"X", "service_rate": "S1"}
test_case = "standard_vs_enforce_BB"
instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/day/enforce_sl_all_hierachical_scenarios.json"
data_dict_path = "../data/dictionary/request_track_data_dictionary.json"

instance_case = Instance(test_case, instance_settings_path, data_dict_path)
# tripdata = ResultTripServiceLevel(instance)


import os.path
import pandas as pd
from pprint import pprint

category_sq_class = pd.api.types.CategoricalDtype(categories=instance_case.dict_sl_class.values(), ordered=True)
category_fleet = pd.api.types.CategoricalDtype(categories=instance_case.dict_fleet.values(), ordered=True)

# Pickup deadline status (Was user chosen to have its pk deadline extended?)
dict_sl_status = instance_case.dict_sl_tier
category_status = pd.api.types.CategoricalDtype(categories=instance_case.dict_sl_tier.values(), ordered=True)
category_segmentation = pd.api.types.CategoricalDtype(categories=instance_case.dict_segmentation.values(), ordered=True)
category_contract_duration = pd.api.types.CategoricalDtype(categories=instance_case.dict_contract_duration, ordered=True)
category_method = pd.api.types.CategoricalDtype(categories=instance_case.dict_method.values(), ordered=True)
category_service_rate = pd.api.types.CategoricalDtype(categories=instance_case.dict_service_rate.values(), ordered=True)


### 2) Loading the instance data

In [35]:
instances_dic = file_util.load_json(instance_settings_path)

# Folder where results will be saved
result_folder = instances_dic["result_folder"]

print("########### INSTANCE SETTINGS ##################################################")
pprint(instances_dic)

########### INSTANCE SETTINGS ##################################################
{'adjacency_matrix_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/network_info/adjacency_matrix.csv',
 'distances_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/distance/dist_matrix_m.csv',
 'durations_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan/distance/dist_matrix_duration_s.csv',
 'hiring_config': [{'name': 'method_hire_from_regional_center'}],
 'instance_description': 'Add auxiliary variable (greater equal)',
 'instance_name': 'SCENARIO',
 'instances_folder': 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/day_enforce_sl/',
 'labels': {'BA': 'batch_duration',
            'CD': 'contract_duration',
            'CS': 'customer_segmentation',
            'CT': 'clear_target_list_every_round',
            'HC': 'maximal_hiring_delay',
            'H

### Get settings from instance name

E.g.:

* Input = `IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR`

* Output =
{allow_many_to_one: True,
allow_service_deterioration: True,
allow_urgent_relocation: True,
allow_vehicle_hiring: True,
batch_duration: 30,
clear_target_list_every_round: True,
contract_duration: 3600,
customer_segmentation: AA,
initial_fleet: 1000,
max_capacity: 06,
max_requests: 1000,
reinsert_targets: True,
time_horizon: 86400}

In [36]:
print(instance_case.dict_sl_class)

{'A': 'Business', 'B': 'Standard', 'C': 'Low-cost'}


### Aggregate results (folder request_track)

* `earliest`
* `id` = 1, 2, 3, ..., #USERS
* `class` = A, B, C
* `delay_pk`
* `ride_delay`
* `pk_time`
* `dp_time`
* `id_from` = Network id
* `id_to` = Network id
* `dist` = trip(id_from, id_to) in seconds
* `service` = {FLEET, FREELANCE}
* `service_level` = {FIRST, SECOND}

In [37]:
import pandas as pd
from pprint import pprint

def get_request_track_dic(path_experiment, name_experiment):

    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)

    # print("Processing experiment file '{}'".format(experiment_file))
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)

    aggfunc = {"delay_pk" : ['mean', 'count', 'max']}

    dfp = df.pivot_table(index="class", columns="service_level", aggfunc=aggfunc, values=["delay_pk"])

    return dfp

### Processing all instances in folder (mean time)

In [38]:
from collections import defaultdict
import math


request_log_folder = result_folder + "request_track/"

# Get all instances in folder
instance_file_names = file_util.read_files_from_folder(request_log_folder)

def process_instance(file_name, instance_attribute, instance_attribute_count, instance_attribute_count_percentage, filter_instance = None, exclude_instance = None):

    instance, extension = file_name.split(".")

    label_setting_dic = instances_dic["labels"]

    # Instance settings
    instance_settings = Instance.get_instance_settings(label_setting_dic, instance)

    # Filter fixed fleet instances
    #if instance_settings.get('service_rate', '-') == '-':
    #    continue

    # Filter instance label
    if filter_instance:
        for i_attribute, i_filter in filter_instance.items():
            if instance_settings[i_attribute] != i_filter:
                print(f"Filtering {instance_settings[i_attribute]} {i_filter}")
                return

    # Filter instance label
    if exclude_instance:
        for i_attribute, i_filter in exclude_instance.items():
            if instance_settings[i_attribute] == i_filter:
                return

    print("  - Processing", instance)

    # Load results
    experiment_file = "{}{}.csv".format(request_log_folder, instance)
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)

    for a in ["service_rate", "contract_duration", "customer_segmentation", "matching"]:
        column = ('', '', a)
        # When no service rate is available, instance refers to baseline
        instance_attribute[column].append(instance_settings.get(a, a+"error"))
        instance_attribute_count_percentage[column].append(instance_settings.get(a, a+"error"))
        instance_attribute_count[column].append(instance_settings.get(a, a+"error"))


    # Convert denied to freelance to show denied under same header of the initial fleet
    df.loc[(df['service']=='DENIED'), 'service'] = 'FLEET'

    # Get total number of requests
    total = len(df)

    for sq_class in ['A', 'B', 'C']:
        for fleet in ['FLEET', 'FREELANCE']:
            for sl_status in ["FIRST", "SECOND"]:
                # Categories leading to max, mean and count
                super_indexes = (instance_case.dict_sl_class[sq_class], instance_case.dict_fleet[fleet], dict_sl_status[sl_status])

                # Filter dataframe
                filter_sq_sl = (df['service'] == fleet) & (df['class'] == sq_class) & (df['service_level'] == sl_status)
                df_filtered = df.loc[filter_sq_sl]["delay_pk"]
                mean_pk = df_filtered.mean()
                #max_pk = df_filtered.max()
                #count = df_filtered.count()

                # Format data
                # mean_pk_min = ("{:.1f}".format(mean_pk/60) if not math.isnan(mean_pk) else "-")
                mean_pk_min = (mean_pk if not math.isnan(mean_pk) else 0)
                #max_pk_min = ("{:.1f}".format(max_pk/60) if not math.isnan(max_pk) else "-")

                #count = ("{:.2%}".format(count/total) if not (math.isnan(count) or count==0) else "-")

                #instance_attribute[(*super_indexes, 'Max.')].append(max_pk_min)
                instance_attribute[super_indexes].append(mean_pk_min)
                #instance_attribute[(*super_indexes, 'Count')].append(count)



                count = df_filtered.count()
                instance_attribute_count[super_indexes].append(count)

                # Format data
                # mean_pk_min = ("{:.1f}".format(mean_pk/60) if not math.isnan(mean_pk) else "-")
                # mean_pk_min = ("{:.1f}".format(int(mean_pk)) if not math.isnan(mean_pk) else "-")
                #max_pk_min = ("{:.1f}".format(max_pk/60) if not math.isnan(max_pk) else "-")

                count = (count/total if not (math.isnan(count) or count==0) else 0)
                instance_attribute_count_percentage[super_indexes].append(count)

                #instance_attribute[(*super_indexes, 'Max.')].append(max_pk_min)
                # instance_attribute[(*super_indexes, 'Mean.')].append(mean_pk_min)

            # Adding total
            subtotal_index = (instance_case.dict_sl_class[sq_class], instance_case.dict_fleet[fleet], 'Total')

            count_fleet = len(df.loc[(df['service'] == fleet) & (df['class'] == sq_class)])
            instance_attribute_count[subtotal_index].append(count_fleet)

            count_fleet = (count_fleet/total if not (math.isnan(count_fleet) or count_fleet==0) else 0)
            instance_attribute_count_percentage[subtotal_index].append(count_fleet)

instance_attribute = defaultdict(list)
instance_attribute_count = defaultdict(list)
instance_attribute_count_percentage = defaultdict(list)

for file_name in instance_file_names:
   process_instance(file_name, instance_attribute, instance_attribute_count, instance_attribute_count_percentage, filter_instance=filter_instance)

Reading 39 files from 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/day_enforce_sl/request_track/'.
  - Processing IN-SCENARIO_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S10_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-SCENARIO_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S8_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-SCENARIO_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_CD-0_SR-S9_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-SCENARIO_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S10_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-SCENARIO_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S8_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-SCENARIO_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-BB_CD-0_SR-S9_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-SCENARIO_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-CC_CD-0_SR-S10_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-SCENARIO_HC-150_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-CC_CD-0_SR-S8_VH_SD_RE-OP_OPT-ERTV
  - Processing IN-SCENARIO_HC

### Raw table (no labels and cell formatting)

In [39]:
# Build data frame from dictionary
raw_table = pd.DataFrame.from_dict(instance_attribute)
raw_table

Business  \
                                                                      Company   
   service_rate contract_duration customer_segmentation  matching    Achieved   
0           S10                 0                    AA      ERTV  104.281943   
1            S8                 0                    AA      ERTV  116.679811   
2            S9                 0                    AA      ERTV  111.750228   
3           S10                 0                    BB      ERTV   88.474359   
4            S8                 0                    BB      ERTV  101.629864   
5            S9                 0                    BB      ERTV   99.596491   
6           S10                 0                    CC      ERTV   86.867516   
7            S8                 0                    CC      ERTV   97.061303   
8            S9                 0                    CC      ERTV   96.418428   
9            S0                 0                    AA    JAVIER   92.147651   
10           S0                 0                    BB    JAVIER   95.456811   
11           S0                 0                    CC    JAVIER   96.953125   
12          S10                 0                    AA      ERTV  118.057706   
13           S8                 0                    AA      ERTV  121.576959   
14           S9                 0                    AA      ERTV  119.183288   
15          S10                 0                    BB      ERTV   95.023364   
16           S8                 0                    BB      ERTV  109.287141   
17           S9                 0                    BB      ERTV  107.836281   
18          S10                 0                    CC      ERTV   93.197836   
19           S8                 0                    CC      ERTV  105.115770   
20           S9                 0                    CC      ERTV  102.872120   
21          S10                 0                    AA  JAVIERSL  121.564187   
22           S8                 0                    AA  JAVIERSL  122.039299   
23           S9                 0                    AA  JAVIERSL  120.479684   
24          S10                 0                    BB  JAVIERSL  122.566161   
25           S8                 0                    BB  JAVIERSL  123.612661   
26           S9                 0                    BB  JAVIERSL  123.630728   
27          S10                 0                    CC  JAVIERSL  124.514835   
28           S8                 0                    CC  JAVIERSL  124.148575   
29           S9                 0                    CC  JAVIERSL  124.622025   
30          S10                 0                    AA      ERTV  120.632618   
31           S8                 0                    AA      ERTV  121.155717   
32           S9                 0                    AA      ERTV  120.731845   
33          S10                 0                    BB      ERTV  100.769270   
34           S8                 0                    BB      ERTV  112.290837   
35           S9                 0                    BB      ERTV  108.467315   
36          S10                 0                    CC      ERTV   94.729312   
37           S8                 0                    CC      ERTV  108.035859   
38           S9                 0                    CC      ERTV  105.379348   

                                       Standard                          \
                 Freelance              Company               Freelance   
      Violated    Achieved Violated    Achieved    Violated    Achieved   
0     0.000000  101.207649      0.0  118.345238    0.000000  104.739899   
1   330.008043  101.908524      0.0  134.139557  367.067227   85.295455   
2   320.224227  104.045971      0.0  124.454957  370.037037  104.975610   
3     0.000000   99.305439      0.0  132.652257    0.000000  104.371225   
4   305.699422  106.563126    349.0  154.403763  351.650980   90.447540   
5   300.687500  103.608696      0.0  146.143309  353.052448   97.5

### Table: What is the service level (pickup delay) of the users lying outside SQ-class service rate?

In [40]:
def create_table(instance_attribute):

    # Build data frame from dictionary
    a = pd.DataFrame.from_dict(instance_attribute)

    key_sr = ('','',  'service_rate') # (S1, S2, S3)
    key_cs = ('','',  'customer_segmentation') # (A, AA, BB, etc.)
    key_cd = ('','',  'contract_duration') #(0, 3600, 18000)
    key_me = ('','',  'matching') #(JAVIER, JAVIERSL, ERTV)

    # Filtering data (only mixed segmentations scenarios will be used)
    # a = a[a[key_cs].isin(["AA", "BB", "CC"])]

    # Renaming data and applying aliases
    a[key_cs] = a[key_cs].map(lambda e:instance_case.dict_segmentation[e])
    a[key_cs] = a[key_cs].astype(category_segmentation)

    a[key_cd] = a[key_cd].map(lambda e:instance_case.dict_contract_duration[e])
    a[key_cd] = a[key_cd].astype(category_contract_duration)

    a[key_sr] = a[key_sr].map(lambda e:instance_case.dict_service_rate[e])
    a[key_sr] = a[key_sr].astype(category_service_rate)

    a[key_me] = a[key_me].map(lambda e:instance_case.dict_method[e])
    a[key_me] = a[key_me].astype(category_method)

    a = a.sort_values(by=[key_cs,key_cd, key_sr, key_me])
    a = a.set_index([key_cs,key_cd, key_sr, key_me])

    # Changing table column index names
    a.index.names = [
        instance_case.headers["user_base"],
        instance_case.headers["contract_duration"],
        instance_case.headers["service_rate"],
        instance_case.headers["method"]
    ]

    # Transforming to minutes
    #a = a.applymap(lambda e: ('{:.2f}'.format(float(e)/60) if e != '-' else e))
    return a


### Table delay

In [41]:
df_delay = create_table(instance_attribute)
df_delay = df_util.format_as_time(df_delay)
df_delay

Business           \
                                                            Company            
                                                           Achieved Violated   
User base Contract duration Service rate Policy                                
B+        NaN               0%           Min. waiting         01:32    04:05   
                            80%          Enforce SL           02:02    03:32   
                                         Enforce SL + Hire    01:56    05:30   
                                         Enforce SL + Hire    02:01    05:01   
                                         Enforce SL + Hire    02:01    04:33   
                            90%          Enforce SL           02:00    03:32   
                                         Enforce SL + Hire    01:51    05:20   
                                         Enforce SL + Hire    01:59    05:09   
                                         Enforce SL + Hire    02:00    04:33   
                            100%         Enforce SL           02:01    03:34   
                                         Enforce SL + Hire    01:44        -   
                                         Enforce SL + Hire    01:58    05:15   
                                         Enforce SL + Hire    02:00    04:50   
S+        NaN               0%           Min. waiting         01:35    05:35   
                            80%          Enforce SL           02:03    04:32   
                                         Enforce SL + Hire    01:41    05:05   
                                         Enforce SL + Hire    01:49    05:23   
                                         Enforce SL + Hire    01:52    05:26   
                            90%          Enforce SL           02:03    04:18   
                                         Enforce SL + Hire    01:39    05:00   
                                         Enforce SL + Hire    01:47    05:00   
                                         Enforce SL + Hire    01:48    05:20   
                            100%         Enforce SL           02:02    04:35   
                                         Enforce SL + Hire    01:28        -   
                                         Enforce SL + Hire    01:35        -   
                                         Enforce SL + Hire    01:40    06:25   
L+        NaN               0%           Min. waiting         01:36    05:27   
                            80%          Enforce SL           02:04    04:40   
                                         Enforce SL + Hire    01:37    04:52   
                                         Enforce SL + Hire    01:45    05:19   
                                         Enforce SL + Hire    01:48    05:31   
                            90%          Enforce SL           02:04    04:39   
                                         Enforce SL + Hire    01:36    04:43   
                                         Enforce SL + Hire    01:42    05:13   
                                         Enforce SL + Hire    01:45    05:25   
                            100%         Enforce SL           02:04    04:41   
                                         Enforce SL + Hire    01:26        -   
                                         Enforce SL + Hire    01:33    04:37   
                                         Enforce SL + Hire    01:34    04:54   

                                                                               \
                                                           Freelance            
                                                            Achieved Violated   
User base Contract duration Service rate Policy                                 
B+        NaN               0%           Min. waiting              -        -   
                            80%          Enforce SL                -        -   
                                         Enforce SL + Hire     01:41        -   
                                         Enforce SL

### Table delay (latex)

In [42]:
print(file_util.df_to_latex(df_delay))

\begin{tabular}{llllllllllllllll}
\toprule
   &     &      &                   & \multicolumn{4}{l}{Business} & \multicolumn{4}{l}{Standard} & \multicolumn{4}{l}{Low-cost} \\
   &     &      &                   & \multicolumn{2}{l}{Company} & \multicolumn{2}{l}{Freelance} & \multicolumn{2}{l}{Company} & \multicolumn{2}{l}{Freelance} & \multicolumn{2}{l}{Company} & \multicolumn{2}{l}{Freelance} \\
   &     &      &                   & Achieved & Violated &  Achieved & Violated & Achieved & Violated &  Achieved & Violated & Achieved & Violated &  Achieved & Violated \\
User base & Contract duration & Service rate & Policy &          &          &           &          &          &          &           &          &          &          &           &          \\
\midrule
\multirow{13}{*}{B+} & \multirow{13}{*}{NaN} & 0\% & Min. waiting &    01:32 &    04:05 &         - &        - &    02:20 &    05:32 &         - &        - &    04:20 &        - &         - &        - \\
   &     & \multirow{

### Table count

In [43]:
df_count = create_table(instance_attribute_count)
df_count

Business           \
                                                            Company            
                                                           Achieved Violated   
User base Contract duration Service rate Policy                                
B+        NaN               0%           Min. waiting          1490     8176   
                            80%          Enforce SL            3766     5900   
                                         Enforce SL + Hire     5072      746   
                                         Enforce SL + Hire     5425     1246   
                                         Enforce SL + Hire     5099     2150   
                            90%          Enforce SL            3667     5999   
                                         Enforce SL + Hire     5493      388   
                                         Enforce SL + Hire     5505     1198   
                                         Enforce SL + Hire     5150     2131   
                            100%         Enforce SL            3630     6036   
                                         Enforce SL + Hire     6136        0   
                                         Enforce SL + Hire     5684     1102   
                                         Enforce SL + Hire     5169     2130   
S+        NaN               0%           Min. waiting           602     1675   
                            80%          Enforce SL            1864      413   
                                         Enforce SL + Hire     1105      173   
                                         Enforce SL + Hire     1602      167   
                                         Enforce SL + Hire     1757      165   
                            90%          Enforce SL            1855      422   
                                         Enforce SL + Hire     1254       80   
                                         Enforce SL + Hire     1753       84   
                                         Enforce SL + Hire     1851       79   
                            100%         Enforce SL            1844      433   
                                         Enforce SL + Hire     1560        0   
                                         Enforce SL + Hire     1926        0   
                                         Enforce SL + Hire     1985        2   
L+        NaN               0%           Min. waiting           704     1573   
                            80%          Enforce SL            1824      453   
                                         Enforce SL + Hire     1044      173   
                                         Enforce SL + Hire     1598      156   
                                         Enforce SL + Hire     1729      156   
                            90%          Enforce SL            1807      470   
                                         Enforce SL + Hire     1183       80   
                                         Enforce SL + Hire     1736       77   
                                         Enforce SL + Hire     1840       71   
                            100%         Enforce SL            1820      457   
                                         Enforce SL + Hire     1570        0   
                                         Enforce SL + Hire     1941        2   
                                         Enforce SL + Hire     2006        2   

                                                                            \
                                                                 Freelance   
                                                           Total  Achieved   
User base Contract duration Service rate Policy                              
B+        NaN               0%           Min. waiting       9666         0   
                            80%          Enforce SL         9666         0   
                                         Enforce SL + Hire  5818      3848   
                                         Enforce SL + Hire  6671      29

### Table count (latex)

In [44]:
print(file_util.df_to_latex(df_count))

\begin{tabular}{llllrrrrrrrrrrrrrrrrrr}
\toprule
   &     &      &                   & \multicolumn{6}{l}{Business} & \multicolumn{6}{l}{Standard} & \multicolumn{6}{l}{Low-cost} \\
   &     &      &                   & \multicolumn{3}{l}{Company} & \multicolumn{3}{l}{Freelance} & \multicolumn{3}{l}{Company} & \multicolumn{3}{l}{Freelance} & \multicolumn{3}{l}{Company} & \multicolumn{3}{l}{Freelance} \\
   &     &      &                   & Achieved & Violated & Total &  Achieved & Violated & Total & Achieved & Violated & Total &  Achieved & Violated & Total & Achieved & Violated & Total &  Achieved & Violated & Total \\
User base & Contract duration & Service rate & Policy &          &          &       &           &          &       &          &          &       &           &          &       &          &          &       &           &          &       \\
\midrule
\multirow{13}{*}{B+} & \multirow{13}{*}{NaN} & 0\% & Min. waiting &     1490 &     8176 &  9666 &         0 &        0 &   

### Table percentage

In [45]:
df_percentage = create_table(instance_attribute_count_percentage)
df_percentage

Business  \
                                                             Company   
                                                            Achieved   
User base Contract duration Service rate Policy                        
B+        NaN               0%           Min. waiting       0.104657   
                            80%          Enforce SL         0.264522   
                                         Enforce SL + Hire  0.356255   
                                         Enforce SL + Hire  0.381049   
                                         Enforce SL + Hire  0.358151   
                            90%          Enforce SL         0.257568   
                                         Enforce SL + Hire  0.385826   
                                         Enforce SL + Hire  0.386669   
                                         Enforce SL + Hire  0.361734   
                            100%         Enforce SL         0.254969   
                                         Enforce SL + Hire  0.430990   
                                         Enforce SL + Hire  0.399241   
                                         Enforce SL + Hire  0.363068   
S+        NaN               0%           Min. waiting       0.042284   
                            80%          Enforce SL         0.130926   
                                         Enforce SL + Hire  0.077615   
                                         Enforce SL + Hire  0.112524   
                                         Enforce SL + Hire  0.123411   
                            90%          Enforce SL         0.130294   
                                         Enforce SL + Hire  0.088080   
                                         Enforce SL + Hire  0.123130   
                                         Enforce SL + Hire  0.130013   
                            100%         Enforce SL         0.129522   
                                         Enforce SL + Hire  0.109574   
                                         Enforce SL + Hire  0.135281   
                                         Enforce SL + Hire  0.139425   
L+        NaN               0%           Min. waiting       0.049449   
                            80%          Enforce SL         0.128117   
                                         Enforce SL + Hire  0.073330   
                                         Enforce SL + Hire  0.112243   
                                         Enforce SL + Hire  0.121444   
                            90%          Enforce SL         0.126923   
                                         Enforce SL + Hire  0.083093   
                                         Enforce SL + Hire  0.121936   
                                         Enforce SL + Hire  0.129241   
                            100%         Enforce SL         0.127836   
                                         Enforce SL + Hire  0.110276   
                                         Enforce SL + Hire  0.136335   
                                         Enforce SL + Hire  0.140900   

                                                                      \
                                                                       
                                                            Violated   
User base Contract duration Service rate Policy                        
B+        NaN               0%           Min. waiting       0.574278   
                            80%          Enforce SL         0.414413   
                                         Enforce SL + Hire  0.052399   
                                         Enforce SL + Hire  0.087518   
                                         Enforce SL + Hire  0.151015   
                            90%          Enforce SL         0.421367   
                                         Enforce SL + Hire  0.027253   
                                         Enforce SL + Hire  0.084147   
                                         Enforce SL + Hire  0.149680   
                           

### Table percentage (formatted)

In [46]:
df_percentage = df_util.format_as_percentage(df_percentage)
df_percentage

Business           \
                                                            Company            
                                                           Achieved Violated   
User base Contract duration Service rate Policy                                
B+        NaN               0%           Min. waiting        10.47%   57.43%   
                            80%          Enforce SL          26.45%   41.44%   
                                         Enforce SL + Hire   35.63%    5.24%   
                                         Enforce SL + Hire   38.10%    8.75%   
                                         Enforce SL + Hire   35.82%   15.10%   
                            90%          Enforce SL          25.76%   42.14%   
                                         Enforce SL + Hire   38.58%    2.73%   
                                         Enforce SL + Hire   38.67%    8.41%   
                                         Enforce SL + Hire   36.17%   14.97%   
                            100%         Enforce SL          25.50%   42.40%   
                                         Enforce SL + Hire   43.10%        -   
                                         Enforce SL + Hire   39.92%    7.74%   
                                         Enforce SL + Hire   36.31%   14.96%   
S+        NaN               0%           Min. waiting         4.23%   11.77%   
                            80%          Enforce SL          13.09%    2.90%   
                                         Enforce SL + Hire    7.76%    1.22%   
                                         Enforce SL + Hire   11.25%    1.17%   
                                         Enforce SL + Hire   12.34%    1.16%   
                            90%          Enforce SL          13.03%    2.96%   
                                         Enforce SL + Hire    8.81%        *   
                                         Enforce SL + Hire   12.31%        *   
                                         Enforce SL + Hire   13.00%        *   
                            100%         Enforce SL          12.95%    3.04%   
                                         Enforce SL + Hire   10.96%        -   
                                         Enforce SL + Hire   13.53%        -   
                                         Enforce SL + Hire   13.94%        *   
L+        NaN               0%           Min. waiting         4.94%   11.05%   
                            80%          Enforce SL          12.81%    3.18%   
                                         Enforce SL + Hire    7.33%    1.22%   
                                         Enforce SL + Hire   11.22%    1.10%   
                                         Enforce SL + Hire   12.14%    1.10%   
                            90%          Enforce SL          12.69%    3.30%   
                                         Enforce SL + Hire    8.31%        *   
                                         Enforce SL + Hire   12.19%        *   
                                         Enforce SL + Hire   12.92%        *   
                            100%         Enforce SL          12.78%    3.21%   
                                         Enforce SL + Hire   11.03%        -   
                                         Enforce SL + Hire   13.63%        *   
                                         Enforce SL + Hire   14.09%        *   

                                                                              \
                                                                   Freelance   
                                                             Total  Achieved   
User base Contract duration Service rate Policy                                
B+        NaN               0%           Min. waiting       67.89%         -   
                            80%          Enforce SL         67.89%         -   
                                         Enforce SL + Hire  40.87%    27.03%   
                                         Enforce SL + Hire

### Table percentage (latex)

In [47]:
print(file_util.df_to_latex(df_percentage))

\begin{tabular}{llllllllllllllllllllll}
\toprule
   &     &      &                   & \multicolumn{6}{l}{Business} & \multicolumn{6}{l}{Standard} & \multicolumn{6}{l}{Low-cost} \\
   &     &      &                   & \multicolumn{3}{l}{Company} & \multicolumn{3}{l}{Freelance} & \multicolumn{3}{l}{Company} & \multicolumn{3}{l}{Freelance} & \multicolumn{3}{l}{Company} & \multicolumn{3}{l}{Freelance} \\
   &     &      &                   & Achieved & Violated &   Total &  Achieved & Violated &   Total & Achieved & Violated &   Total &  Achieved & Violated &   Total & Achieved & Violated &   Total &  Achieved & Violated &   Total \\
User base & Contract duration & Service rate & Policy &          &          &         &           &          &         &          &          &         &           &          &         &          &          &         &           &          &         \\
\midrule
\multirow{13}{*}{B+} & \multirow{13}{*}{NaN} & 0\% & Min. waiting &   10.47\% &   57.43\% &  67.89\

In [48]:
fleet = list(instance_case.dict_fleet.values())
fleet.remove("Denied")
tiers = list(instance_case.dict_sl_tier.values())
tiers.remove("Rejected")
classes = list(instance_case.dict_sl_class.values())

tuples = [
        (a, b, c) for a in classes
        for b in fleet for c in tiers
]

# Removing total
no_total= df_percentage[tuples]
no_total


Business           \
                                                            Company            
                                                           Achieved Violated   
User base Contract duration Service rate Policy                                
B+        NaN               0%           Min. waiting        10.47%   57.43%   
                            80%          Enforce SL          26.45%   41.44%   
                                         Enforce SL + Hire   35.63%    5.24%   
                                         Enforce SL + Hire   38.10%    8.75%   
                                         Enforce SL + Hire   35.82%   15.10%   
                            90%          Enforce SL          25.76%   42.14%   
                                         Enforce SL + Hire   38.58%    2.73%   
                                         Enforce SL + Hire   38.67%    8.41%   
                                         Enforce SL + Hire   36.17%   14.97%   
                            100%         Enforce SL          25.50%   42.40%   
                                         Enforce SL + Hire   43.10%        -   
                                         Enforce SL + Hire   39.92%    7.74%   
                                         Enforce SL + Hire   36.31%   14.96%   
S+        NaN               0%           Min. waiting         4.23%   11.77%   
                            80%          Enforce SL          13.09%    2.90%   
                                         Enforce SL + Hire    7.76%    1.22%   
                                         Enforce SL + Hire   11.25%    1.17%   
                                         Enforce SL + Hire   12.34%    1.16%   
                            90%          Enforce SL          13.03%    2.96%   
                                         Enforce SL + Hire    8.81%        *   
                                         Enforce SL + Hire   12.31%        *   
                                         Enforce SL + Hire   13.00%        *   
                            100%         Enforce SL          12.95%    3.04%   
                                         Enforce SL + Hire   10.96%        -   
                                         Enforce SL + Hire   13.53%        -   
                                         Enforce SL + Hire   13.94%        *   
L+        NaN               0%           Min. waiting         4.94%   11.05%   
                            80%          Enforce SL          12.81%    3.18%   
                                         Enforce SL + Hire    7.33%    1.22%   
                                         Enforce SL + Hire   11.22%    1.10%   
                                         Enforce SL + Hire   12.14%    1.10%   
                            90%          Enforce SL          12.69%    3.30%   
                                         Enforce SL + Hire    8.31%        *   
                                         Enforce SL + Hire   12.19%        *   
                                         Enforce SL + Hire   12.92%        *   
                            100%         Enforce SL          12.78%    3.21%   
                                         Enforce SL + Hire   11.03%        -   
                                         Enforce SL + Hire   13.63%        *   
                                         Enforce SL + Hire   14.09%        *   

                                                                               \
                                                           Freelance            
                                                            Achieved Violated   
User base Contract duration Service rate Policy                                 
B+        NaN               0%           Min. waiting              -        -   
                            80%          Enforce SL                -        -   
                                         Enforce SL + Hire    27.03%        -   
                                         Enforce SL